In [1]:
from google.colab import drive
from datetime import datetime

import matplotlib.pyplot as plt
import os
import pandas as pd
import sqlite3
from pandas import DataFrame as df
import numpy as np
from pandas import read_csv as rc


In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/HOP for 10K+/Data/raw')
dire=os.getcwd()
print(os.listdir())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
['p_kraken.csv', 'p_luno.csv', 'markets_fixed.db', 'p_luno_19Jul.csv']


In [3]:
pluno=rc('p_luno.csv')
pkraken=rc('p_kraken.csv')

def clean_prices(df):
    df = df[(df['best_bid'] > 100) & (df['best_ask'] > 100) & (df['best_bid'] < 100000) & (df['best_ask'] < 100000)]
    df = df[df['best_ask'] - df['best_bid'] > -5]
    df['timestamp_rounded'] = pd.to_datetime(df['timestamp'].round(0), unit='s')
    df = df.set_index('timestamp_rounded')
    return df


In [4]:
pk=clean_prices(pkraken)
pl=clean_prices(pluno)

In [16]:
def makearray(pindex):
  timearray=[]
  y=str(pindex[0])[5:16]
  timearray.append(y)
  i0=0
  indexarrayL=[i0]

  for x in pindex:
    if y!=str(x)[5:16]:
      timearray.append(str(x)[5:16])
      indexarrayL.append(i0)
      y=str(x)[5:16]
    i0+=1
  return [indexarrayL,timearray]

In [17]:
indexarrayL, timearrayL = makearray(pl.index)
indexarrayK, timearrayK = makearray(pk.index)

In [77]:
mergeddf=df(columns=['Time', 'kBid', 'kAsk','lBid','lAsk'])
for i in indexarrayL:
  i=pl.index[i]
  if type(pl.loc[i,'best_bid'])==pd.core.series.Series:
    mergeddf=mergeddf.append({'Time': str(i)[5:16], 'lBid':pl.loc[i,'best_bid'].values[0],'lAsk':pl.loc[i,'best_ask'].values[0]}, ignore_index=True)
  else:
    mergeddf=mergeddf.append({'Time': str(i)[5:16], 'lBid':pl.loc[i,'best_bid'],'lAsk':pl.loc[i,'best_ask']}, ignore_index=True)

In [92]:
# i=indexarrayL[0]
# i=pl.index[i]
# type(pl.loc[i,'best_bid'])
# type(pl.loc[i,'best_bid'])==pd.core.series.Series
mergeddf=mergeddf.set_index('Time')

In [136]:
for i in set(timearrayL).intersection(set(timearrayK)):
  for k in indexarrayK:
    if str(pk.index[k])[5:16]==i:
      break
  k=pk.index[k]
  if type(pk.loc[k,'best_bid'])==pd.core.series.Series:
    mergeddf.loc[i][0:2]=[pk.loc[k,'best_bid'].values[0],pk.loc[k,'best_ask'].values[0] ]
  else:
    mergeddf.loc[i][0:2]=[pk.loc[k,'best_bid'],pk.loc[k,'best_ask'] ]
#  mergeddf.loc[i][0:2]=[,]

In [137]:
#mergeddf.loc['07-10 12:47'][0:2]=[1,3]
mergeddf

,kBid,kAsk,lBid,lAsk
Time,,,,
07-10 12:47,8145.77,8150.56,8151.00,8159.90
07-10 12:48,8146.95,8150.56,8151.00,8160.00
07-10 12:49,8146.49,8150.56,8151.00,8160.00
07-10 12:50,8149.73,8150.56,8151.10,8160.00
07-10 12:51,8153.53,8150.56,8151.10,8160.00
...,...,...,...,...
07-14 20:57,8140.97,8145.41,8139.28,8152.11
07-14 20:58,8140.97,8145.22,8140.33,8152.11
07-14 20:59,8143.00,8144.78,8140.44,8152.11


In [138]:
mergeddf.to_csv('/content/drive/My Drive/HOP for 10K+/Data/cooked/merged.csv')